In [1]:
import os
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
import re
from geopy.geocoders import Nominatim
import numpy as np
import folium
import pandas as pd
import time

In [2]:
def load_settings():
    with open('settings.json', encoding='utf8') as f:
        settings = json.load(f)
    return settings

In [3]:
settings = load_settings()

In [7]:
def get_html(
    settings,
    page_init = str(1),

):
    #Set vars
    url_init = settings['url_init']['url']
    user_agent = settings['scraper_user_agent']
    file_name = settings['file_name']
    filter_start = settings['url_init']['size_filter']['min']
    filter_end = settings['url_init']['size_filter']['max']
    filter_step = settings['url_init']['size_filter']['step']
    
    # Set up the browser
    options = webdriver.ChromeOptions()
    options.add_argument('--headless=new')  # Run the browser in headless mode (without GUI)
    #user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.50 Safari/537.36'
    options.add_argument(f'user-agent={user_agent}')
    
    driver=webdriver.Chrome(options=options)
    
    start = filter_start
    href_links_final = []
    
    while start < filter_end:
        try:
            driver=webdriver.Chrome(options=options)
            # Navigate to the URL
            url_filt = url_init.format(
                start,
                start+filter_step
            )+page_init
            driver.get(url_filt)

            # Get the HTML content
            #html_content = driver.page_source

            #Total number of pages
            pages =  driver.find_element('xpath','.//span[@class = "text-battleship"]').get_attribute("textContent")
            page_num = int(re.findall(r'\d+', pages)[0])

        finally:
            # Close the browser
            driver.quit()

        #link collector
        href_links_all = []

        for i in range(page_num): #page_num
            #Get updated url
            driver=webdriver.Chrome(options=options)
            url = url_init.format(
                start,
                start+filter_step
            )+ str(i+1)
            driver.get(url)

            #Links
            ids = driver.find_elements(by=By.XPATH, value="//a[@data-listing-id]")
            href_links = ["https://ingatlan.com/" + e.get_attribute("data-listing-id") for e in ids]
            href_links_all.extend(href_links)

            # Close the browser
            driver.quit()

            print(f"{i+1}/{page_num} is done")
        
        href_links_final.extend(href_links_all) 
        
        start+=filter_step
        print(f'{url_init.format(start,start+filter_step)} finished')
        
    
    with open(r'{}'.format(file_name), 'w') as fp:
        for item in href_links_final:
            fp.write("%s\n" % item)
    

    return href_links_final

In [8]:
#Possible urr error
href_links_all = get_html(settings = settings)

1/8 is done
2/8 is done
3/8 is done
4/8 is done
5/8 is done
6/8 is done
7/8 is done
8/8 is done
https://ingatlan.com/lista/elado+lakas+80-82-m2+budapest-budai-oldal+2-szoba-felett+70-120-mFt?page= finished
1/9 is done
2/9 is done
3/9 is done
4/9 is done
5/9 is done
6/9 is done
7/9 is done
8/9 is done
9/9 is done
https://ingatlan.com/lista/elado+lakas+82-84-m2+budapest-budai-oldal+2-szoba-felett+70-120-mFt?page= finished


In [9]:
len(href_links_all)

326

In [12]:
data = pd.read_json("data/all_data_buda_backup.json").T

<ipython-input-12-e713e096c7c6>:1: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  data = pd.read_json("data/all_data_buda_backup.json").T


In [13]:
data.reset_index(inplace = True)

In [17]:
data.rename(columns={'index':'scrape_time'}, inplace = True)

In [31]:
def create_map(settings):
    
    data = pd.read_json(settings['data_file_name']).T
    
    #Get scraping time
    data.reset_index(inplace = True)
    data.rename(columns={'index':'scrape_time'}, inplace = True)
    
    # Get data filters from settings
    filter_dict = settings['filters']

    # Use filter to reduce datasize
    for column, conditions in filter_dict.items():
        data = data[(data[column] >= conditions['min']) & (data[column] < conditions['max'])].copy()
    
    #Create map    
    m2 = folium.Map([47.5025527, 19.0321423], zoom_start=11.5)
    
    for i,row in data.iterrows():
        coordinates = row["Koordináták"]
        url = row['url']
        price = row["Ár_millió"]
        size = row["Alapterület"]
        rooms = row["Szobák"]
        scraping_time = row["scrape_time"]

        if coordinates is not None:
            iframe = folium.IFrame(
                f'''
                URL: <a href={url}>{url}</a><br> Price (mill): {price}<br> Size (m2): {size}<br> \
                Rooms (full-sized): {rooms}<br> Scraping time: {scraping_time}<br>
                '''
            )
            popup = folium.Popup(iframe,
                                 min_width=230,
                                 max_width=230)
            folium.Marker(
                location=row["Koordináták"],
                tooltip="Details",
                popup=popup,
                icon=folium.Icon(color="blue"),
            ).add_to(m2)

    return m2.save(settings["map_file_name"])

In [32]:
create_map(settings)

<ipython-input-31-fbef97f7970b>:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  data = pd.read_json("data/all_data_buda_backup.json").T


In [2]:
import json
 
# Opening JSON file
f = open("data/all_data_buda.json", encoding = 'utf-8')
 
# returns JSON object as 
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list

# Closing file
f.close()

In [3]:
from functions import enrich_data

In [4]:
data_new = enrich_data(data)

Enriching data with coordinates..
All done


In [5]:
save_file = open("data/all_data_buda.json", "w", encoding='utf8')  
json.dump(all_data, save_file, indent = 6, ensure_ascii=False)  
save_file.close()

NameError: name 'all_data' is not defined

In [23]:
for i,v in data.items():
    if "Pontos_cím" not in data[i]:
        print(i)
        print(data[i])

In [25]:
def enrich_data(all_data):
    print("Enriching data with coordinates..")
    
    geolocator = Nominatim(user_agent="realestate-locator")
    
    for i,v in all_data.items():
        if "Pontos_cím" not in all_data[i]:
            try:
                location = geolocator.geocode(v["Cím"])
                all_data[i]["Pontos_cím"] = location.address
                all_data[i]["Koordináták"] = (location.latitude, location.longitude)
            except AttributeError:
                all_data[i]["Pontos_cím"] = np.nan
                all_data[i]["Koordináták"] = np.nan
        else:
            pass
        
    print("All done")
    
    return all_data

In [26]:
data_new = enrich_data(data)

Enriching data with coordinates..
All done


In [21]:
len(data_new)

732

In [22]:
len(data)

732